In [47]:
def map_concat_columns(row,q_col):
    if len(q_col) == 3:
        act_col = q_col[0]
        product_col = q_col[1]
        usaha = q_col[2]
        aktivitas = row[act_col] if pd.notnull(row[act_col]) else ''
        ## if end with dot, remove it

        produk = row[product_col] if pd.notnull(row[product_col]) else ''

        usaha = row[usaha] if pd.notnull(row[usaha]) else ''

        if aktivitas.endswith('.'):
            aktivitas = aktivitas[:-1]
            ## if end with dot, remove it
        if produk.endswith('.'):
            produk = produk[:-1]

        if usaha.endswith('.'):
            usaha = usaha[:-1]
        aktivitas=clean_text(aktivitas)
        produk=clean_text(produk)
        usaha=clean_text(usaha)
        #print (f"{aktivitas}.{produk}.{usaha}.")
        return f"{aktivitas}.{produk}.{usaha}."
    else:print("error.. kolom aktivitas,produk,usaha tidak sesuai")


def split_label(series):
    label = series.iloc[0]  # fix FutureWarning
    label_split = label.split('] ')
    code = label_split[0].removeprefix('[')
    label = label_split[1]
    return code, label
def clean_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9.,:;()\-/ ]', ' ', text)  # hanya simbol dasar
    text = re.sub(r'\s+', ' ', text)  # hapus spasi berlebih
    text = re.sub(r'([.,])([^\s])', r'\1 \2', text)
    text = text.replace('.', '').replace('"', '')  # hapus titik dan kutip ganda
    text = text.strip().strip('"')  # tambahan ini memastikan hapus kutip di awal/akhir
    return text

def cleaning_dataset(df,label_col,q_col):
    print("start cleaning dataset")
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    
    df = df.dropna(subset=label_col)  # kamu lupa simpan hasil dropna
    print("remove missing label complete")
    df_new = df[q_col+label_col].copy()  # fix SettingWithCopyWarning

    q_df = df_new[q_col]
    print("assign Question to dataframe complete")
    text_description = q_df.apply(lambda row: map_concat_columns(row,q_col), axis=1)
    print("concat question complete")
    
    if len(label_col) == 2:
        kbli_df = df_new[[label_col[0]]]
        temp_kbli_code_label = kbli_df.apply(split_label, axis=1)
        kbli_codes = temp_kbli_code_label.apply(lambda x: x[0])
        kbli_labels = temp_kbli_code_label.apply(lambda x: x[1])

        kbji_df = df_new[[label_col[1]]]
        temp_kbji_code_label = kbji_df.apply(split_label, axis=1)
        kbji_codes = temp_kbji_code_label.apply(lambda x: x[0])
        kbji_labels = temp_kbji_code_label.apply(lambda x: x[1])

    df_new['text_description'] = text_description
    df_new['kbli_code'] = kbli_codes
    df_new['kbli_label'] = kbli_labels
    df_new['kbji_code'] = kbji_codes
    df_new['kbji_label'] = kbji_labels

    clean_df = df_new[['text_description','kbli_code','kbli_label','kbji_code','kbji_label']]
    return clean_df


In [48]:
import pandas as pd
import re
df_path="../dataset/raw/raw_sak_kalteng.xlsx"

## read excel
sheets=pd.read_excel(df_path,sheet_name=None)

# Filter sheet yang berakhiran '_u' saja
filtered_sheets = {name: df for name, df in sheets.items() if name.endswith('_u')}


# Dictionary untuk simpan hasil
cleaned_data = {}

# Proses semua sheet
for sheet_name, df in filtered_sheets.items():
    print(f"Processing sheet: {sheet_name}")
    try:
        print(f"Original DataFrame shape: {df.shape}")
        print(f"sheet_name: {sheet_name}")
        cleaned_df = cleaning_dataset(df,label_col=['b5r15a_label', 'b5r15b_label'],q_col=['b5r13a', 'b5r13b','b5r13c'])
        print(f"Cleaned DataFrame shape: {cleaned_df.shape}")
        cleaned_data[sheet_name] = cleaned_df
    except Exception as e:
        print(f"Error in sheet {sheet_name}: {e}")

Processing sheet: 6201_u
Original DataFrame shape: (973, 35)
sheet_name: 6201_u
start cleaning dataset
remove missing label complete
assign Question to dataframe complete
concat question complete
Cleaned DataFrame shape: (972, 5)
Processing sheet: 6202_u
Original DataFrame shape: (999, 34)
sheet_name: 6202_u
start cleaning dataset
remove missing label complete
assign Question to dataframe complete
concat question complete
Cleaned DataFrame shape: (868, 5)
Processing sheet: 6203_u
Original DataFrame shape: (999, 34)
sheet_name: 6203_u
start cleaning dataset
remove missing label complete
assign Question to dataframe complete
concat question complete
Cleaned DataFrame shape: (983, 5)
Processing sheet: 6204_u
Original DataFrame shape: (999, 34)
sheet_name: 6204_u
start cleaning dataset
remove missing label complete
assign Question to dataframe complete
concat question complete
Cleaned DataFrame shape: (816, 5)
Processing sheet: 6205_u
Original DataFrame shape: (848, 34)
sheet_name: 6205_u


In [49]:
cleaned_data['6201_u']

,text_description,kbli_code,kbli_label,kbji_code,kbji_label
0,karyawan panen kelapa sawit di kebun pt sampoe...,01262,PERKEBUNAN BUAH KELAPA SAWIT,6113,PEKERJA PERTANIAN TANAMAN KEBUN BIBIT DAN TANA...
1,"karyawan perawatan (memupuk, nyemprot, menebas...",01262,PERKEBUNAN BUAH KELAPA SAWIT,6113,PEKERJA PERTANIAN TANAMAN KEBUN BIBIT DAN TANA...
2,ketua koperasi kelompok tani mitra terpadu iv....,01262,PERKEBUNAN BUAH KELAPA SAWIT,1211,MANAJER KEUANGAN
3,"merawat, membersihkan, memupuk kebun sawit mil...",01262,PERKEBUNAN BUAH KELAPA SAWIT,6112,PEKERJA PERTANIAN TANAMAN TAHUNAN DAN TANAMAN ...
4,pemilik dan penjual hasil kebun kelapa sawit.b...,01262,PERKEBUNAN BUAH KELAPA SAWIT,6112,PEKERJA PERTANIAN TANAMAN TAHUNAN DAN TANAMAN ...
...,...,...,...,...,...
968,menanam sawit.buah sawit.pertanian milik sendiri.,01262,PERKEBUNAN BUAH KELAPA SAWIT,6111,PEKERJA PERTANIAN TANAMAN PANGAN DAN TANAMAN S...
969,menjaga kios bbm.bbm(bahan bakar minyak).jual ...,47302,"PERDAGANGAN ECERAN BAHAN BAKAR MINYAK, BAHAN B...",5221,PEMILIK TOKO
970,tambang emas tradisional.emas.non pertanian.,07301,PERTAMBANGAN EMAS DAN PERAK,8112,OPERATOR MESIN PENGOLAHAN BIJIH MINERAL DAN BATU
971,mengutip berondolan sawit.berondol sawit.perta...,01262,PERKEBUNAN BUAH KELAPA SAWIT,9622,PEKERJA SERABUTAN


In [50]:
import csv
# Gabungkan semua sheet yang berhasil jadi satu DataFrame
combined_df = pd.concat(cleaned_data.values(), ignore_index=True)

# Simpan ke file baru (opsional)
combined_df.to_csv("../dataset/clean/df_kbli_subset.csv",quoting=csv.QUOTE_ALL, index=False)
